# Distributed Protocols

A *distributed protocol* is a specification for a computation performed jointly by several communicating *parties*. In this book, we'll be concerned with *synchronous* protocols, in which communication between the parties occurs at pre-specified times.

## Choreographic Programming and PyChor

In this book, we'll implement protocols using the PyChor library. PyChor is a Python library for [*choreographic programming*](https://en.wikipedia.org/wiki/Choreographic_programming), a programming paradigm in which a single global program specifies the computations and synchronous communications of all parties.

You can install PyChor using `pip`:

```
pip install pychor
```

The PyChor library enables the implementation of protocols as Python programs in a choreographic style. PyChor adds *located values*---values known to a single party participating in the protocol---as special kinds of Python values. For example, the following choreography consists of a client and server, and the client knows that the value of the variable `x` is 5 (`x` is *located* at the client).

In [1]:
import pychor
client = pychor.Party('client')
server = pychor.Party('server')

with pychor.LocalBackend():
    x = 5@client
    print(x)

5@{Party(name='client')}


Here, the `with` block starts a new choreography using the *local backend*. The local backend simulates the execution of the protocol on a single computer, without using any actual network communication. The `@` symbol is used in PyChor to indicate that a value should be located at a particular location, so `5@client` is the value 5 located at the client.

To send a value to another party, we can use the `send` operator. The expression `x.send(p, q)` sends the value of `x` from party `p` to party `q`. After this operation, *both* `p` and `q` will know the value of `x`---it is *located at both parties*. For example, the following protocol constructs a value located at the client, then the client sends that value to the server.

In [2]:
with pychor.LocalBackend():
    x = 5@client
    x.send(client, server)
    print(x)

5@{Party(name='client'), Party(name='server')}


One example of a simple but useful protocol is one that performs *outsourced computation*---for example, the client may have a long list of numbers and want to compute the sum, but may not have sufficient processing power to do so. The client can send the numbers to the server, and the server can compute the sum and send it back to the client:

In [3]:
with pychor.LocalBackend():
    # initialize the data on the client side
    xs = [1,2,3,4,5,6,7,8,9]@client

    # send the data from the client to the server
    xs.send(client, server)

    # server sums up the results
    result_server = pychor.locally(sum, xs.only(server))
    print('Server result:', result_server)

    # server sends results to the client
    result_server.send(server, client)
    print('Final result:', result_server)

Server result: 45@{Party(name='server')}
Final result: 45@{Party(name='client'), Party(name='server')}


Here, the expression `pychor.locally(sum, xs)` performs some *local computation* at the server. The `locally` method allows choreographies to use standard Python functions that don't work for located values, but requires all of the inputs to be located at the party performing the computation. The `only` method restricts the `xs` object to be located at the server *only* - otherwise, both the client and the server will perform the sum at the same time, resulting in duplicated work.

This is (on purpose) not a very useful instance of outsourced computation - the computation being performed is just a sum, and the client might as well do it themselves. We'll use this as a stand-in for more advanced computation for now, and we'll see examples of more complicated settings later on.

# Homework: extend this to $n$ servers and $m$ clients